# Acceso a BD relacionales SQLite desde Python
_Antonio Sarasa, Enrique Martín_

Python tiene integrada una base de datos relacional denominada SQLite. Podéis encontrar la documentación detallada en https://docs.python.org/3/library/sqlite3.html.

Para crear una base de datos:

* En primer lugar hay que realizar una conexión con el servidor de la base de datos. Esto se hace mediante la función **connect**. 
* En el caso de sqlite3 sólo se necesita pasar como parámetro una cadena con la ruta al archivo en el que guardar los datos de la base de datos, o bien la cadena “:memory:” para utilizar la memoria RAM en lugar de un fichero en disco.

Vamos a crear una base de datos denominada “Biblioteca”

In [1]:
import sqlite3

conn = sqlite3.connect("biblioteca.sqlite3")
conn

La función connect devuelve un objeto de tipo Connection que representa la conexión con la base de datos almacenada en el archivo biblioteca.sqlite3 del directorio actual. Si el archivo no existe, se creará nuevo. 

Las distintas operaciones que se pueden realizar con la base de datos se realizan a través de un objeto Cursor. Para crear este objeto se utiliza el método cursor() del objeto Connection.


In [2]:
cur = conn.cursor()
cur

Una vez que tenemos el cursor, se pueden ejecutar comandos sobre el contenido de la base de datos, usando el método execute() que toma como argumento una cadena con el código SQL a ejecutar.

Por ejemplo se va a crear una tabla llamada Libros con una columna de texto llamada “Título” y otra columna de enteros llamada “prestamos”. Además antes de crear la tabla la vamos a eliminar para asegurarse que no existe ya en la base de datos.

In [3]:
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS Libros")
cur.execute("CREATE TABLE Libros (titulo TEXT, ejemplares INTEGER)")
cur.close()

__No obstante, muchos de los métodos del cursos están disponibles también en la conexión, por lo que no es necesario crear explícitamente el objeto Cursor (que muchas veces es superfluo)__

In [4]:
conn.execute("DROP TABLE IF EXISTS Libros")
conn.execute("CREATE TABLE Libros (titulo TEXT, ejemplares INTEGER)")

Una vez creada la tabla Libros se pueden guardar datos usando una llamada a execute() con el comando SQL INSERT. Este comando indica qué tabla se va a utilizar y luego define una fila nueva, enumerando los campos a incluir y seguidos por los valores (VALUES) que se desean colocar en esa fila. 


In [5]:
conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES ('El Quijote', 20)")
conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES ('El Escarabajo de oro', 15)")

Otra forma de insertar consiste en especificar como signos de interrogación  (?,?) los valores para indicar que serán pasados como una tupla en el segundo parámetro de la llamada a execute(). Podéis encontrar más información sobre el uso de los marcadores "?" en https://docs.python.org/3/library/sqlite3.html#sqlite3-placeholders.


**IMPORTANTE: lo veremos con detalle más adelante en el curso, pero es IMPRESCINDIBLE USAR MARCADORES al ejecutar sentencias SQL desde vuestro programa para mitigar las vulnerabilidades de inyección SQL.**

In [6]:
conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote', 20))
conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))

Si la base de datos soporta transacciones y están activadas, y además la característica de auto-commit está desactivada, será necesario llamar al método commit de la conexion para que se lleven a cabo las operaciones definidas en la transacción.

Si en estas circunstancias se usara una herramienta externa para comprobar el contenido de la base de datos sin hacer primero el commit aparecería entonces con una base de datos vacía. Sin embargo si se consulta desde Python parecería que se han llevado a cabo los cambios, aunque no es así. 

In [7]:
cur = conn.execute("SELECT * FROM Libros")
for res in cur.fetchall():
    print(res)

('El Quijote', 20)
('El Escarabajo de oro', 15)
('El Quijote', 20)
('El Escarabajo de oro', 15)


In [8]:
conn.commit()

Si la  base de datos soporta la característica de rollback  entonces se puede cancelar la transacción actual con el método rollback de la conexión. Si la base de datos no soporta rollback, entonces al llamar a este método producirá una excepción.

Cuando se quieren insertar múltiples filas en una sola operación se puede usar el método executemany y proporcionar como argumento la secuencia de filas que se quieren insertar como una lista. Como resultado se llama al método execute una vez por cada fila.

In [9]:
cur = conn.executemany("INSERT INTO Libros(titulo, ejemplares) VALUES (?,?)", [('El Quijote',20),('El Escarabajo de oro', 15)])
print(cur.rowcount)
conn.commit()

2


El mismo efecto se podría haber conseguido utilizando un bucle sobre una lista y el método execute().

In [10]:
entradas = [('El Quijote',20), ('El Escarabajo de oro', 15)]
for fila in entradas:
    conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", fila)
conn.commit()

In [11]:
def init_db(conn):
    conn.execute("DROP TABLE IF EXISTS Libros")
    conn.execute("CREATE TABLE Libros (titulo TEXT, ejemplares INTEGER)")
    conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)", ('El Quijote', 20))
    conn.execute("INSERT INTO Libros (titulo, ejemplares) VALUES (?,?)",  ('El Escarabajo de oro', 15))
    conn.commit()

Para realizar consultas a la base de datos también se utiliza el método execute tomando como argumento una cadena que represente una sentencia SELECT de SQL.

Cuando se realiza una consulta, el cursor no lee todos los datos de la base de datos cuando se ejecuta la sentencia SELECT sino que los datos serán leídos a medida que se pidan las filas.

Para consultar las tuplas resultantes de la sentencia SQL se puede llamar a los métodos de cursor __fetchone, fetchmany o fetchall__ o usar el objeto cursor como un iterador.

En los siguientes ejemplos primero se insertaran dos filas en la tabla con INSERT y luego se usará commit() para forzar que los datos sean escritos en el archivo de la base de datos. Después se usará el comando SELECT para recuperar las filas que se acaban de insertar en la tabla, y en cada ejemplo se usará una forma distinta de pedir las filas recuperadas. Al final del programa se  ejecuta el comando DELETE para borrar las filas que se acaban de crear, y por último se ejecuta un commit() para forzar a los datos a ser eliminados de la base de datos.

En este ejemplo se usa fetchall que recupera una lista de las filas que hay en la tabla.

In [12]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT titulo, ejemplares FROM Libros")
print(cur.fetchall())
conn.commit()

Libros
[('El Quijote', 20), ('El Escarabajo de oro', 15)]


También es posible iterar sobre las tuplas obtenidas con fetchall:

In [13]:
init_db(conn)
print ("Libros")
cur = conn.execute("SELECT titulo, ejemplares FROM Libros")
for fila in cur.fetchall():
    print(fila)
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


También es posible recuperar los valores de las tuplas utilizando tuplas en el bucle, si conocemos el número de columnas que tiene cada fila.

In [14]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT titulo, ejemplares FROM Libros")
for (titulo, ejemplares) in cur.fetchall():
    print (f"Título: {titulo} --> {ejemplares}")
conn.commit()

Libros
Título: El Quijote --> 20
Título: El Escarabajo de oro --> 15


En el siguiente ejemplo se recupera una única columna de la tabla y luego se usa fetchall:

In [15]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT titulo FROM Libros")
titulos = cur.fetchall()
print(titulos)
conn.commit()

Libros
[('El Quijote',), ('El Escarabajo de oro',)]


En este último caso se podría formatear para que en vez de devolver una tupla con un solo valor devolviera una lista con los valores recuperados usando comprensión de listas

In [16]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT * FROM Libros")
titulos = [rec[0] for rec in cur.fetchall()]
print(titulos)
conn.commit()

Libros
['El Quijote', 'El Escarabajo de oro']


Si las tablas que se usan son muy grandes es mejor no pedir todas las filas mediante fetchall, usando otros métodos alternativos como fetchone que devuelve la siguiente tupla del conjunto resultado o None si no existen más 

In [17]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT * FROM Libros")
while True:
    fila = cur.fetchone()
    if not fila: 
        break
    print(fila)
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Otra alternativa a fetchall es el método fetchmany que devuelve el número de tuplas indicado por el entero pasado como parámetro o bien el número indicado por el atributo Cursor.arraysize si no se pasa ningún parámetro( por defecto vale 1).

In [18]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT * FROM Libros")
while True:
    filas = cur.fetchmany(2)
    if not filas: 
        break
    for fila in filas:
        print(fila)
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Observar que una vez que se han recuperado todas las filas con fetchall, fetchone o fetchmany, si se quieren volver a recuperar las filas sería necesario realizar una nueva llamada a execute con la sentencia SELECT dado que se pierden una vez recuperadas.

Alternativamente a los métodos anteriores, también es posible iterar sobre el cursor con el que se ha realizado la consulta.

In [19]:
init_db(conn)
print("Libros")
cur = conn.execute("SELECT * FROM Libros")
for resultado in cur:
    print(resultado)
conn.commit()

Libros
('El Quijote', 20)
('El Escarabajo de oro', 15)


Para realizar actualizaciones o borrados también se usa el método execute del objeto cursor o la conexión. En el siguiente ejemplo se va actualizar la columna “ejemplares” de la fila correspondiente al libro con título “El quijote” y se va a rellenar con el valor 22.

Si se hace la consulta para ver lo que hay en la base de datos:

In [20]:
init_db(conn)
conn.execute('UPDATE Libros set ejemplares=? WHERE titulo=?',[22,"El Quijote"])
conn.commit()
cur = conn.execute("SELECT * FROM Libros")
for (titulo, ejemplar) in cur.fetchall():
    print ("Titulo:", titulo)
    print ("Ejemplares:", ejemplar)
    print('------')
conn.commit()

Titulo: El Quijote
Ejemplares: 22
------
Titulo: El Escarabajo de oro
Ejemplares: 15
------


En el siguiente ejemplo se va eliminar la fila correspondiente al libro con título “El quijote”.

In [21]:
init_db(conn)
conn.execute('DELETE FROM Libros WHERE titulo=?',["El Quijote"])
conn.commit()
cur = conn.execute("SELECT * FROM Libros")
for (titulo, ejemplar) in cur.fetchall():
    print ("Titulo:", titulo)
    print ("Ejemplares:", ejemplar)
    print('------')
conn.commit()

Titulo: El Escarabajo de oro
Ejemplares: 15
------


Se pueden realizar operaciones de columna cuando se ejecuta una sentencia SELECT. En el siguiente ejemplo se van a sumar los ejemplares de todos los libros almacenados en la base de datos.

In [22]:
init_db(conn)
cur = conn.execute('SELECT sum(ejemplares) FROM Libros')
print(cur.fetchall())
conn.commit()

[(35,)]


## Ejemplo completo de uso de SQLite desde Python

Se quiere implementar un programa que guarde información de los amigos que una persona tiene en Twitter y de las relaciones que tienen a su vez estos amigos entre sí. Esta información se almacenará en una base de datos.

En primer lugar se fija la estructura de la base de datos que se va a utilizar:

* Se creará una tabla llamada Personas que almacenará la información de las cuentas de Twitter.
* Se creará una tabla llamada Seguimientos que almacenará las relaciones que existen entre las personas.

La tabla Personas dispondrá de 3 columnas:

* Un id que actuará como clave primaría de la tabla.
* El nombre de usuario de la cuenta de Twitter.

In [23]:
import sqlite3
conn = sqlite3.connect("twitter.sqlite3")

conn.execute('''DROP TABLE IF EXISTS Personas''')
conn.execute('''CREATE TABLE Personas(
                 id INTEGER PRIMARY KEY, 
                 nombre TEXT UNIQUE)''')
conn.commit()

La tabla Seguimiento dispondrá de 2 columnas que contendrán id´s de usuarios registrados en la tabla Personas y que representarán una relación de seguimiento entre ambos usuarios con un sentido definido.

In [24]:
conn.execute('''DROP TABLE IF EXISTS Seguimientos''')
conn.execute('''CREATE TABLE Seguimientos(
                 desde_id INTEGER REFERENCES Personas(id), 
                 hacia_id INTEGER REFERENCES Personas(id),
                 
                 PRIMARY KEY(desde_id, hacia_id)
                 CHECK (desde_id != hacia_id)
                 )''')

Observar que la combinación de los dos números de cada fila de la tabla Seguimientos es clave primaria, lo que evita que se cometan errores como añadir la misma relación entre las mismas personas más de una vez. Además, los id que usemos deben existir en la tabla Personas previamente.

In [25]:
personas = [ (0, 'pepe'), 
             (1, 'ana'),
             (2, 'eva'),
             (3, 'luis'),
             (4, 'raquel'),  
           ]
conn.executemany('INSERT INTO Personas VALUES (?,?)', personas)
conn.commit()

In [26]:
# Esta inserción lanzaría una excepción porque se viola la clave primaria
# conn.execute('INSERT INTO Personas VALUES (4, "enrique")') # IntegrityError: UNIQUE constraint failed: Personas.id

In [27]:
personas = [ (0, 1), (0, 2), (0, 3), (0, 4),
             (1, 0), (1, 4),
             (2, 3),
             (3, 0), (3, 1), (3, 4),
           ]
conn.executemany('INSERT INTO Seguimientos VALUES (?,?)', personas)
conn.commit()

In [28]:
# IntegrityError: CHECK constraint failed: desde_id != hacia_id
# conn.execute('INSERT INTO Seguimientos VALUES (0, 0)')

In [29]:
# Total de personas
cur = conn.execute('SELECT COUNT(*) FROM Personas')
npersonas, = cur.fetchone()
npersonas

5

In [30]:
# Total de relaciones de seguimiento
cur = conn .execute('SELECT COUNT(*) FROM Seguimientos')
nseg, = cur.fetchone()
nseg

10

In [31]:
# Número de personas a las que sigue el usuario de ID=0
cur.execute('''SELECT COUNT(*) 
               FROM Seguimientos
               WHERE desde_id = 0''')
namigos, = cur.fetchone()
namigos

4

In [32]:
# Número de personas a las que sigue la usuaria 'raquel'
cur = conn.execute('''SELECT COUNT(*) 
                      FROM Personas P JOIN Seguimientos S ON P.id = S.desde_id
                      WHERE P.nombre = "raquel"''')
namigos, = cur.fetchone()
namigos

0

In [33]:
# Número de personas a las que sigue el usuario 'pepe'
cur = conn.execute('''SELECT COUNT(*) 
                      FROM Personas P JOIN Seguimientos S ON P.id = S.desde_id
                      WHERE P.nombre = "pepe"''')
namigos, = cur.fetchone()
namigos

4

In [34]:
# Por cada persona, número de personas a las que siguen 
query = '''
SELECT P.id, P.nombre, COUNT(*) AS 'Num_Amigos'
FROM Personas P JOIN Seguimientos S ON P.id = S.desde_id
GROUP BY P.id, P.nombre
ORDER BY COUNT(*) ASC
'''

namigos = conn.execute(query).fetchall()
namigos

[(2, 'eva', 1), (1, 'ana', 2), (3, 'luis', 3), (0, 'pepe', 4)]

In [35]:
# Por cada persona, número de personas a las que siguen *contando también los usuarios sin seguimientos*
query = '''
SELECT P.id, P.nombre, COUNT(desde_id) AS 'Num_Amigos'
FROM Personas P LEFT JOIN Seguimientos S ON P.id = S.desde_id
GROUP BY P.id, P.nombre
ORDER BY COUNT(desde_id) DESC
'''

namigos = conn.execute(query).fetchall()
namigos

[(0, 'pepe', 4),
 (3, 'luis', 3),
 (1, 'ana', 2),
 (2, 'eva', 1),
 (4, 'raquel', 0)]

In [36]:
# Usuario más seguido, en caso de empate el de ID más pequeño
query = '''
SELECT P.id, P.nombre, COUNT(S.hacia_id) AS 'Num_seguidores'
FROM Personas P JOIN Seguimientos S ON P.id = S.hacia_id
GROUP BY P.id, P.nombre
ORDER BY COUNT(S.hacia_id) DESC, P.id ASC
LIMIT 1'''

nseguidores, = conn.execute(query).fetchall()
nseguidores

(4, 'raquel', 3)

In [37]:
# Al realizar una consulta, cur.description incluye la descripción de las columnas
conn.execute(query).description

(('id', None, None, None, None, None, None),
 ('nombre', None, None, None, None, None, None),
 ('Num_seguidores', None, None, None, None, None, None))

Sí, devuelve tuplas de 7 elementos aunque los últimos 6 son `None` (https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.description). Esto es así para cumplir con la especificación Python de acceso a bases de datos **DB-API 2.0** (https://peps.python.org/pep-0249/). Según esta especificación, la descripción del cursor (https://peps.python.org/pep-0249/#description) debe devolver 7 valores: name, type_code, display_size, internal_size, precision, scale, null_ok. Pero la implementación de SQLite en Python ignora todos menos el nombre de columna.

In [38]:
# Usando cur.description se puede generar un diccionario en lugar de tuplas
cur = conn.execute('SELECT * FROM Personas')
print(cur.description)
[{desc[0]: row[pos] for pos, desc in enumerate(cur.description)} for row in cur]
# Explicación: a) genero una lista por comprensión para cada fila (row) generada por la consulta (cur)
#              b) para cada fila, genero un diccionario por comprensión recorriendo todos sus campos según la descripción

(('id', None, None, None, None, None, None), ('nombre', None, None, None, None, None, None))


[{'id': 0, 'nombre': 'pepe'},
 {'id': 1, 'nombre': 'ana'},
 {'id': 2, 'nombre': 'eva'},
 {'id': 3, 'nombre': 'luis'},
 {'id': 4, 'nombre': 'raquel'}]